## Deeplabcut prediction of mouse position


We use 2 dlc models to predict the position of the mouse in the `arena_top.cropped` and `positrack2` videos.

In [1]:
%load_ext autoreload
%autoreload 2
%run setup_project.py
import os.path
import shutil
import cv2
from autopipy.dlcObjectDetectors import MouseLEDsDetector
from autopipy.cvObjectDetectors import ArenaDetector

/home/kilo/repo/analysis_jingjie/jingjie/autopi_mec_jingjie/autopi_mec/mouse_and_objects_position_tracking/setup_project.py:769: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()
/home/kilo/repo/analysis_jingjie/jingjie/autopi_mec_jingjie/autopi_mec/mouse_and_objects_position_tracking/setup_project.py:797: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()


Loading DLC 2.2.3...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [2]:
projectName, dataPath, dlcModelPath, myProject, sSessions= setup_project_session_lists()

creating myProject, an autopipy.project object
Project name: autopi_mec
dataPath: /adata/projects/autopi_mec
dlcModelPath: /adata/models
Reading /adata/projects/autopi_mec/sessionList_invalid
We have 1 testing sessions in the list
spikeA.Kilosort_session objects are in sSessions


## Check that we have the files we need

In [3]:
for ses in sSessions:
    ses.load_parameters_from_files() 
    for t in ses.trial_names:
        pfn = ses.path+"/"+t+".positrack2"
        vidfn = ses.path+"/"+t+".mp4"
        for fn in [pfn,vidfn]:
            if not os.path.exists(fn):
                print(ses.name,fn,"is missing") 

In [4]:
for ses in myProject.sessionList:
    fn = ses.fileNames['arena_top.cropped.avi']
    if not os.path.exists(fn):
                print(ses.name,fn,"is missing") 

# Process the positrack2 files


We process the positrack2 video offline. The model takes the full size image and with an old GPU runs at 31 Hz. This is about real-time performance.

In [5]:
for i,ses in enumerate(sSessions):
    ses.load_parameters_from_files() 
    dlcDone=True
    for t in ses.trial_names:
        pfn = ses.path+"/"+t+".positrack2"
        vidfn = ses.path+"/"+t+".mp4"
        out = ses.path+"/"+t+"DLC_resnet50_positrack2MouseLEDs_540_840Sep29shuffle1_300000.h5"
        if not os.path.exists(out):
            dlcDone=False
    
    if dlcDone == False:
        print(i,ses.name)   

In [6]:
modelDirectory="/adata/models/positrack2MouseLEDs_540_840-Allen-2022-09-29"
configFile=modelDirectory+"/config.yaml"
mouseD = MouseLEDsDetector(pathConfigFile=configFile)

In [7]:
ses = sSessions[0]
ses.name

'mn8578-30112021-0107'

In [8]:
for ses in sSessions[22:23]:
    for t in ses.trial_names[4:5]:
        pfn = ses.path+"/"+t+".positrack2"
        vidfn = ses.path+"/"+t+".mp4"
        print(pfn)
        mouseD.inferenceVideo(pathVideoFile=vidfn,overwrite=True)
        mouseD.labelVideo(pathVideoFile=vidfn)

Check a few label videos to make sure the quality is ok. It looked pretty good. 

# Process the arena_top.cropped.avi video to get the animal position



In [9]:
modelDirectory="/adata/models/arenaTopMouseLEDs_480_480-Allen-2022-09-29"
configFile=modelDirectory+"/config.yaml"
mouseD = MouseLEDsDetector(pathConfigFile=configFile)

In [10]:
for i,ses in enumerate(myProject.sessionList):
    
    croppedVideoFile = ses.fileNames['arena_top.cropped.avi']
    out = ses.path+"/"+ses.name+".arena_top.cropped" + "DLC_resnet50_arenaTopMouseLEDs_480_480Sep29shuffle1_700000.csv"
    #print(out)
    if not os.path.exists(out):
        print(i,ses.name)   

In [11]:
for ses in myProject.sessionList[0:45]:
    croppedVideoFile = ses.fileNames['arena_top.cropped.avi']
    mouseD.inferenceVideo(pathVideoFile=croppedVideoFile,overwrite=False)
    #mouseD.labelVideo(pathVideoFile=croppedVideoFile)

Start date and time = 13/02/2024 10:16:55
Running dlc.analyze_video on /adata/projects/autopi_mec/mn8578/mn8578-30112021-0107/mn8578-30112021-0107.arena_top.cropped.avi
Using snapshot-700000 for model /adata/models/arenaTopMouseLEDs_480_480-Allen-2022-09-29/dlc-models/iteration-0/arenaTopMouseLEDs_480_480Sep29-trainset95shuffle1


/home/kilo/miniconda3/envs/spikeA/lib/python3.11/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2024-02-13 10:16:57.865998: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-13 10:16:57.869620: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-13 10:16:57.869784

Starting to analyze %  /adata/projects/autopi_mec/mn8578/mn8578-30112021-0107/mn8578-30112021-0107.arena_top.cropped.avi
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
dlc.loadPositionData() read data from /adata/projects/autopi_mec/mn8578/mn8578-30112021-0107/mn8578-30112021-0107.arena_top.croppedDLC_resnet50_arenaTopMouseLEDs_480_480Sep29shuffle1_700000.h5
dlc.loadPositionData() read data from /adata/projects/autopi_mec/mn8578/mn8578-30112021-0107/mn8578-30112021-0107.arena_top.croppedDLC_resnet50_arenaTopMouseLEDs_480_480Sep29shuffle1_700000.h5
Saving position data to /adata/projects/autopi_mec/mn8578/mn8578-30112021-0107/mn8578-30112021-0107.arena_top.croppedDLC_resnet50_arenaTopMouseLEDs_480_480Sep29shuffle1_700000.csv
End d